In [29]:
# For preprocessing
from keras.preprocessing.image import ImageDataGenerator

# For sequential CNN
from keras.models import Sequential
from keras import layers
from keras.optimizers import Adam
from keras.losses import binary_crossentropy

# For Inception
from keras import Input
from keras import Model

# Plotting
from keras.utils import plot_model
import matplotlib.pyplot as plt
# MacOS matplotlib kernel issue
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [31]:
dataset = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_gen = dataset.flow_from_directory('../Datasets/kagglecatsanddogs_3367a/PetImages/', 
                                        classes=['Cat', 'Dog'], 
                                        target_size = (128, 128), 
                                        batch_size = 32, 
                                        color_mode='rgb', subset='training')
valid_gen = dataset.flow_from_directory('../Datasets/kagglecatsanddogs_3367a/PetImages/', 
                                        classes=['Cat', 'Dog'], 
                                        target_size = (128, 128), 
                                        batch_size = 32, 
                                        color_mode='rgb', subset='validation')

# Note that Cat/666.jpg and Dog/11702.jpg images are corrupted in the dataset

Found 20000 images belonging to 2 classes.
Found 4998 images belonging to 2 classes.


In [37]:
sequential_cnn = Sequential()
sequential_cnn.add(layers.Conv2D(32, kernel_size=3, activation='relu', input_shape=train_gen.image_shape))
sequential_cnn.add(layers.MaxPool2D((2,2)))
sequential_cnn.add(layers.Conv2D(64, kernel_size=3, activation='relu'))
sequential_cnn.add(layers.MaxPool2D((2,2)))
sequential_cnn.add(layers.Conv2D(128, kernel_size=3, activation='relu'))
sequential_cnn.add(layers.MaxPool2D((2,2)))
sequential_cnn.add(layers.Conv2D(256, kernel_size=3, activation='relu'))
sequential_cnn.add(layers.MaxPool2D((2,2)))
sequential_cnn.add(layers.Conv2D(512, kernel_size=3, activation='relu'))
sequential_cnn.add(layers.Flatten())
sequential_cnn.add(layers.Dense(256, activation='relu'))
sequential_cnn.add(layers.Dense(2, activation='sigmoid'))

In [38]:
sequential_cnn.compile(optimizer=Adam(lr=0.001), loss=binary_crossentropy, metrics=['acc'])

In [39]:
sequential_cnn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 126, 126, 32)      896       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 61, 61, 64)        18496     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 30, 30, 64)        0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 28, 28, 128)       73856     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 12, 12, 256)       295168    
__________

In [40]:
plot_model(sequential_cnn, to_file="sequential_cnn.png", show_shapes=True, show_layer_names=True)

In [41]:
sequential_fit = sequential_cnn.fit_generator(train_gen, steps_per_epoch=100, epochs=30, 
                                              validation_data=valid_gen, validation_steps=50)

Epoch 1/30
100/100 [==============================] - 47s 465ms/step - loss: 0.6910 - acc: 0.5259 - val_loss: 0.6911 - val_acc: 0.5038
Epoch 2/30
100/100 [==============================] - 45s 448ms/step - loss: 0.6672 - acc: 0.5919 - val_loss: 0.6536 - val_acc: 0.6344
Epoch 3/30
 15/100 [===>..........................] - ETA: 31s - loss: 0.6689 - acc: 0.6115

KeyboardInterrupt: 

In [42]:
valid_acc = sequential_fit.evaluate_generator(valid_gen, steps=50)
print(valid_acc)

NameError: name 'sequential_fit' is not defined

In [44]:
inception_input = Input(shape=train_gen.image_shape)

block1 = layers.Conv2D(64, kernel_size=3, padding='same', activation='relu')(inception_input)
block1 = layers.MaxPooling2D((2,2))(block1)
block1 = layers.Conv2D(64, kernel_size=3, padding='same', activation='relu')(block1)
block1 = layers.Conv2D(32, kernel_size=3, padding='same', activation='relu')(block1)
block1 = layers.MaxPooling2D((2,2))(block1)
block1 = layers.Conv2D(16, kernel_size=3, padding='same', activation='relu')(block1)

block2 = layers.Conv2D(64, kernel_size=3, padding='same', activation='relu')(inception_input)
block2 = layers.Conv2D(128, kernel_size=3, padding='same', activation='relu')(block2)
block2 = layers.MaxPooling2D((2,2))(block2)
block2 = layers.Conv2D(64, kernel_size=3, padding='same', activation='relu')(block2)
block2 = layers.MaxPooling2D((2,2))(block2)
block2 = layers.Conv2D(16, kernel_size=3, padding='same', activation='relu')(block2)

inception_output = layers.concatenate([block1, block2], axis=1)
inception_output = layers.Flatten()(inception_output)
inception_output = layers.Dense(2, activation='sigmoid')(inception_output)

inception_model = Model(inputs=inception_input, outputs=inception_output)

In [45]:
inception_model.compile(optimizer=Adam(lr=0.0001), loss=binary_crossentropy, metrics=['acc'])

In [46]:
inception_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
conv2d_30 (Conv2D)              (None, 128, 128, 64) 1792        input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_34 (Conv2D)              (None, 128, 128, 64) 1792        input_2[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_17 (MaxPooling2D) (None, 64, 64, 64)   0           conv2d_30[0][0]                  
__________________________________________________________________________________________________
conv2d_35 

In [47]:
plot_model(inception_model, to_file="inception_cnn.png", show_shapes=True, show_layer_names=True)

In [49]:
inception_fit = inception_model.fit_generator(train_gen, steps_per_epoch=100, epochs=30, 
                                              validation_data=valid_gen, validation_steps=50)

Epoch 1/30
100/100 [==============================] - 273s 3s/step - loss: 0.6926 - acc: 0.5200 - val_loss: 0.6878 - val_acc: 0.5197
Epoch 2/30
 99/100 [============================>.] - ETA: 2s - loss: 0.6818 - acc: 0.5720

KeyboardInterrupt: 

In [ ]:
valid_acc = sequential_fit.evaluate_generator(valid_gen, steps=50)
print(valid_acc)